In [20]:
from __future__ import absolute_import
import numpy as nm
from sfepy.discrete.fem import Mesh
from sfepy.discrete.fem.meshio import UserMeshIO
import csv
import numpy as np

In [21]:
def mesh_hook(mesh, mode):
    """
    Generate the 1D mesh.
    """
    if mode == 'read':
        n_nod = 101

        coors = nm.linspace(0.0, 1.0, n_nod).reshape((n_nod, 1))
        conn = nm.arange(n_nod, dtype=nm.int32).repeat(2)[1:-1].reshape((-1, 2))
        mat_ids = nm.zeros(n_nod - 1, dtype=nm.int32)
        descs = ['1_2']

        mesh = Mesh.from_data('laplace_1d', coors, None,
                              [conn], [mat_ids], descs)
        return mesh

    elif mode == 'write':
        pass

In [22]:
def get_coef(ts, coors, mode=None, **kwargs):
    if mode == 'qp':
        x = coors[:, 0]

        val = 0.1 + nm.sin(2 * nm.pi * x)**2
        val.shape = (coors.shape[0], 1, 1)

        return {'val' : val}

In [23]:
filename_mesh = UserMeshIO(mesh_hook)

filename_mesh

UserMeshIO

In [24]:
n_nod = 101

coors = nm.linspace(0.0, 1.0, n_nod).reshape((n_nod, 1))
conn = nm.arange(n_nod, dtype=nm.int32).repeat(2)[1:-1].reshape((-1, 2))
mat_ids = nm.zeros(n_nod - 1, dtype=nm.int32)
descs = ['1_2']

mesh = Mesh.from_data('laplace_1d', coors, None,
                        [conn], [mat_ids], descs)

mesh

Mesh:laplace_1d

In [25]:
# Read boundary condition data from CSV
def read_temp_and_hcoeff_from_csv(filename="t_o_and_h_o.csv"):
    """Read temperature and heat transfer coefficient data from CSV file."""
    t_o, h_o = [], []
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            if len(row) >= 2:
                t_o.append(float(row[0].strip()))
                h_o.append(float(row[1].strip()))
    return t_o, h_o

t_o, h_o = read_temp_and_hcoeff_from_csv()


In [26]:
h_o[len(h_o)-1]

17.2

In [27]:
def get_coef(ts, coors, mode=None, **kwargs):
    if mode == 'qp':
        x = coors[:, 0]

        val = 0.1 + nm.sin(2 * nm.pi * x)**2
        val.shape = (coors.shape[0], 1, 1)

        return {'val' : val}

In [28]:
coors = np.array([[0.0], [0.5], [1.0]])  # Example coordinates
result = get_coef(None, coors, mode='qp')
print(result)


{'val': array([[[0.1]],

       [[0.1]],

       [[0.1]]])}


In [29]:
def read_temp_and_hcoeff_from_csv(filename="t_o_and_h_o.csv"):
    """Read temperature and heat transfer coefficient data from CSV file."""
    t_o, h_o = [], []
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            if len(row) >= 2:
                t_o.append(float(row[0].strip()))
                h_o.append(float(row[1].strip()))
    return t_o, h_o

t_o, h_o = read_temp_and_hcoeff_from_csv()
nr_hour = len(t_o)  # Number of hours

In [30]:
def get_bc_values(ts, coor, mode=None, region_name=None, **kwargs):
    """
    Time-dependent boundary condition values.
    """
    if mode != 'qp': return {}
    # Boundary conditions
    t_i = 0.0         # Inner temperature [°C]
    h_i = 99.75       # Inner heat transfer coefficient [W/m2K]
    
    hour = ts.time / 3600  # Current hour
    hour_idx = min(int(hour), nr_hour - 1)  # Index for current hour
    
    nqp = coor.shape[0]
    val = nm.zeros((nqp, 1, 2), dtype=nm.float64)  # h and T_inf
    
    if region_name == 'Gamma_Left':
        # Left boundary (outer surface)
        val[:, 0, 0] = h_o[hour_idx]  # h value
        val[:, 0, 1] = t_o[hour_idx]   # T_inf value
    elif region_name == 'Gamma_Right':
        # Right boundary (inner surface)
        val[:, 0, 0] = h_i  # h value
        val[:, 0, 1] = t_i   # T_inf value
    
    return {'val': val}

In [31]:
class TimeStruct:
    def __init__(self, time):
        self.time = time

ts = TimeStruct(7200)  # 2 hours into the simulation
coor = np.array([[0.0], [0.5], [1.0]])  # Example coordinates
region_name = 'Gamma_Left'  # Specify the boundary

result = get_bc_values(ts, coor, mode='qp', region_name=region_name)
print(result)

{'val': array([[[31.90734191, -4.1       ]],

       [[31.90734191, -4.1       ]],

       [[31.90734191, -4.1       ]]])}


In [32]:
nr_hour = len(t_o)  # Number of hours
print(nr_hour)
# Time parameters
dt = 10.0  # Time step in seconds
nh = int(3600 / dt)  # Number of time steps per hour
print(nh)
total_time = nr_hour * 3600  # Total simulation time in seconds
print(total_time)
n_step = nr_hour * nh  # Total number of time steps
print(n_step)

3672
360
13219200
1321920
